In [138]:
import time
import random
import selenium

from pprint import pprint

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException, TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains






In [142]:
username = "K.Dixon1@lse.ac.uk"
password = ""

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import StaleElementReferenceException, TimeoutException

def scrape_exam_papers(driver):
    list = []

    def get_elements(css_selector, retries=3):
        """Utility function to get elements with retry on stale element exception."""
        for attempt in range(retries):
            try:
                return WebDriverWait(driver, 10).until(EC.visibility_of_all_elements_located((By.CSS_SELECTOR, css_selector)))
            except StaleElementReferenceException:
                if attempt < retries - 1:
                    continue
                else:
                    raise

    def get_element(css_selector, retries=3):
        """Utility function to get a single element with retry on stale element exception."""
        for attempt in range(retries):
            try:
                return WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.CSS_SELECTOR, css_selector)))
            except StaleElementReferenceException:
                if attempt < retries - 1:
                    continue
                else:
                    raise

    try:
        boxes = get_elements(".margin-bottom-small > prm-gallery-collection")

        for m in range(len(boxes)):
            try:
                # Re-find the boxes element to avoid stale reference
                boxes = get_elements(".margin-bottom-small > prm-gallery-collection")
                box = boxes[m]
                details = {}

                # Extract title and URL
                title = get_element(f".margin-bottom-small > prm-gallery-collection:nth-child({m+1}) > a:nth-child(1) > div:nth-child(1) > div:nth-child(1) > h3:nth-child(2) > span:nth-child(1)").text
                subject_page_url = box.find_element(By.TAG_NAME, "a").get_attribute("href")
                box.click()

                containers = get_elements(f".is-grid-view > prm-gallery-item")

                for i in range(len(containers)):
                    try:
                        # Re-find the containers element to avoid stale reference
                        containers = get_elements(f".is-grid-view > prm-gallery-item")
                        container = containers[i]

                        course_name = container.find_element(By.CLASS_NAME, "item-title").text
                        type = container.find_element(By.CSS_SELECTOR, ".is-grid-view > prm-gallery-item:nth-child(1) > div:nth-child(2) > div:nth-child(2) > span:nth-child(1) > span:nth-child(1)").text
                        container.click()

                        exams = get_elements("md-list-item.md-3-line:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(2) > a:nth-child(1)")

                        for n in range(len(exams)):
                            exam = exams[n]
                            exam_year = exam.text
                            pdf_link = exam.get_attribute("href")

                            details = {
                                "name": title,
                                "subject_page_url": subject_page_url,
                                "course_name": course_name,
                                "type": type,
                                "exam_year": exam_year,
                                "pdf_link": pdf_link
                            }
                            list.append(details)
                            print(details)

                            exit_button = get_element("button.md-icon-button:nth-child(4)")
                            exit_button.click()

                        back_button = get_element("prm-collection-navigation-breadcrumbs-item.layout-align-start-center:nth-child(2) > a:nth-child(1)")
                        actions = ActionChains(driver)
                        actions.move_to_element(back_button).click().perform()

                    except (StaleElementReferenceException, TimeoutException) as e:
                        print(f"Container processing error: {e}")
                        continue

            except (StaleElementReferenceException, TimeoutException) as e:
                print(f"Box processing error: {e}")
                continue

    except TimeoutException as e:
        print(f"Initial element fetching error: {e}")

    return list


In [141]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# ...

driver = webdriver.Firefox()
driver.get('https://www.lse.ac.uk/library')

# Let the user actually see something!
WebDriverWait(driver, 5).until(EC.visibility_of_element_located((By.ID, 'Loginto'))).click()

# Let the user actually see something!
WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, '/html/body/div[3]/div/div/h3[1]/a'))).click()

# Let the user actually see something!
WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.NAME, 'loginfmt'))).send_keys(username)

# Let the user actually see something!
next_button = WebDriverWait(driver, 5).until(EC.visibility_of_element_located((By.ID, 'idSIButton9')))
if next_button:
    next_button.click()
else:
    print("No next button found")

# Let the user actually see something!
WebDriverWait(driver, 5).until(EC.visibility_of_element_located((By.NAME, 'passwd'))).send_keys(password)

# Let the user actually see something!
next_button = WebDriverWait(driver, 5).until(EC.visibility_of_element_located((By.ID, 'idSIButton9')))
next_button.click()

# Wait until the screen changes, this can be a variable length of time though
try:
    # Wait until the element is visible
    element = WebDriverWait(driver, 300).until(EC.visibility_of_element_located((By.ID, 'idBtn_Back')))
    # Act on the element as soon as it becomes visible
    element.click()
except TimeoutException:
    print("The element did not appear within the time limit")

# Let the user actually see something!
WebDriverWait(driver, 5).until(EC.visibility_of_element_located((By.CSS_SELECTOR, 'div.grid-item:nth-child(1) > button:nth-child(1)'))).click()

new_url = WebDriverWait(driver, 10).until(EC.visibility_of_all_elements_located((By.CSS_SELECTOR, '#mainMenu > div:nth-child(4) > a:nth-child(1)')))[0].get_attribute("href")

# Let the user actually see something!
WebDriverWait(driver, 5).until(EC.visibility_of_element_located((By.CSS_SELECTOR, '#mainMenu > div:nth-child(4) > a:nth-child(1)'))).click()

#the driver must now be on the new page

WebDriverWait(driver, 10).until(EC.number_of_windows_to_be(2))
new_window = driver.window_handles[-1]
driver.switch_to.window(new_window)

scrape_exam_papers(driver)








{'name': 'Accounting', 'subject_page_url': 'https://librarysearch.lse.ac.uk/discovery/collectionDiscovery?vid=44LSE_INST%3A44LSE_VU1&collectionId=81235317140002021&lang=en', 'course_name': 'AC103 - Elements of Management Accounting, Financial Management and Financial Institutions', 'type': 'EXAM PAPER', 'exam_year': '2019', 'pdf_link': 'https://librarysearch.lse.ac.uk/discovery/delivery/44LSE_INST:44LSE_VU1/12235315290002021'}
Container processing error: Message: 

Container processing error: Message: 

Container processing error: Message: 

Container processing error: Message: 

Container processing error: Message: 

Container processing error: Message: 

Container processing error: Message: 

Container processing error: Message: 

Container processing error: Message: 

Container processing error: Message: 

Container processing error: Message: 

Container processing error: Message: 

Container processing error: Message: 

Container processing error: Message: 

Container processing er

KeyboardInterrupt: 